In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [4]:
df = pd.read_csv('../Data/block_data_ready_cleaned.csv')
df.dropna(inplace=True)

# Define your features and target variable
features = ['gas_used', 'base_fee_per_gas_in_eth', 'validator_is_registered_with_relay',
            'is_fb_builder', 'sandwiches_count', 'liquidations_count', 
            'sandwiched_swaps_count', 'arbitrages_count', 'hour_of_day', 
            'block_fullness', 'total_gas_fees', 'base_fee_change_percentage', 
            'network_demand']
target = 'proposer_total_reward_in_eth'

# Prepare the data
X = df[features]
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Define the parameter grid to search
param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.5, 0.1, 0.2],
    'colsample_bytree': [0.3, 0.7],
    'alpha': [5, 10]
}

# Initialize XGBoost Regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Initialize Grid Search
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=3, verbose=2)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print best parameters and corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_search.best_score_)))

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.9s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.8s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.6s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100; total time=   1.0s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100; total time=   1.0s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100; total time=   1.0s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=150; total time=   1.6s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=150; total time=   1.5s
[CV] END alpha=5, colsample_bytree=0.3, learning_rate=0.01, 

In [22]:
# Initialize XGBoost Regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.01,
                          max_depth=3, alpha=5, n_estimators=150)

# Train the model
xg_reg.fit(X_train, y_train)

# Make predictions
y_pred = xg_reg.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'XGBoost RMSE: {rmse}')

XGBoost RMSE: 0.7262827862497012
